In [1]:
# configuration cell

no_epochs = 30

act_fn_dict = {
    'conv1':'relu',
    'conv2':'relu',
    'conv3':'relu',
    'conv4':'relu',
    'conv5':'relu',
    'fc1':'relu'
}

kernel_size_list = [3,3,3,3,3]
no_kernel_list = [32,32,64,64,128]
dropout_list = [0,0,0.5]
fc1_nodes = 1024
no_classes = 10
lr = 0.0001
lr_schedule = 0.5 # per 10 epochs half the learning rate

In [2]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
import numpy as np
import shutil
import os
import wandb
from util import*

In [3]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()
if use_cuda == True:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [4]:
modelName = 'Best_CNN_5Layers_iNaturalist_Images'

In [5]:
ckp_path = "./Best_CNN_5Layers_iNaturalist/checkpoint/current_checkpoint.pt"
best_ckp_path = "./Best_CNN_5Layers_iNaturalist/best_model/best_model.pt"

In [ ]:
wandb.init(project=modelName)

In [7]:
batch_size = 64
datasetTrain, datasetVal, datasetTest = load_datasets()
loaders = data_loader(datasetTrain, datasetVal, datasetTest, batch_size)

In [8]:
def act_fn(act_name):
    if act_name=="relu":
        return nn.ReLU(inplace=True)
    
    elif act_name=="sigmoid":
        return nn.Sigmoid(inplace=True)
    
    elif act_name=="tanh":
        return nn.Tanh(inplace==True)

In [9]:
class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size = 3 ,BN=True , NL="relu", stride = 1, padding = 0):
        super(conv_block, self).__init__()
        self.BN=BN
        self.NL=NL
        k = kernel_size
        
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size = k, stride = stride, padding = padding, bias=False)
        
        if self.BN==True:
            self.bn = nn.BatchNorm2d(out_channels, eps=0.001)
            
        self.act = act_fn(NL)
        
    def forward(self, x):
        x = self.conv(x)
        
        if self.BN==True:
            x = self.bn(x)
        
        return self.act(x)

In [10]:
class fc_block(nn.Module):
    def __init__(self, in_channels, out_channels, BN=False , NL="relu"):
        super(fc_block, self).__init__()
        self.BN=BN
        self.NL=NL
        self.fc = nn.Linear(in_channels, out_channels)
        
        if self.BN==True:
            self.bn = nn.BatchNorm2d(out_channels, eps=0.001)
            
        self.act = act_fn(NL)
        
    def forward(self, x):
        x = self.fc(x)
        
        if self.BN==True:
            x = self.bn(x)
        
        x = self.act(x)
        
        return x

In [11]:
def get_fc_in(input_dim, kernel_size_list, no_kernel_list):
    H = input_dim
    fc_in = H - kernel_size_list[0] + 1 # conv1
    fc_in = (fc_in - 2) //2  + 1 # max pool 1
    fc_in = fc_in - kernel_size_list[1] + 1 # conv2
    fc_in = (fc_in - 2) //2  + 1 # max pool 2
    fc_in = fc_in - kernel_size_list[2] + 1 #conv3
    fc_in = (fc_in - 2) //2  + 1 # max pool 3
    fc_in = fc_in - kernel_size_list[3] + 1 #conv4
    fc_in = (fc_in - 2) //2  + 1 # max pool 4
    fc_in = fc_in - kernel_size_list[4] + 1 #conv5
    fc_in = (fc_in - 2) //2  + 1 # max pool 5
    #print(fc_in)
    return fc_in * fc_in * no_kernel_list[4]

In [12]:
class CNN_5layer(nn.Module):
    def __init__(self, kernel_size_list, no_kernel_list, act_fn_dict, dropout_list, fc1_nodes, no_classes):
        super(CNN_5layer, self).__init__()
        self.dropout_list = dropout_list
        self.input_dim = 224
        #self.input_dim = 128
        self.conv1 = conv_block(3, no_kernel_list[0], kernel_size=kernel_size_list[0], BN=False, NL=act_fn_dict['conv1'])
        self.maxpool1 = nn.MaxPool2d((2, 2))
        self.conv2 = conv_block(no_kernel_list[0], no_kernel_list[1], kernel_size=kernel_size_list[1], BN=True, NL=act_fn_dict['conv2'])
        self.maxpool2 = nn.MaxPool2d((2, 2))
        
        if self.dropout_list[0]!=0:
            self.dropout1 = nn.Dropout(dropout_list[0])

        self.conv3 = conv_block(no_kernel_list[1], no_kernel_list[2], kernel_size=kernel_size_list[2], BN=True, NL=act_fn_dict['conv3'])
        self.maxpool3 = nn.MaxPool2d((2, 2))
        self.conv4 = conv_block(no_kernel_list[2], no_kernel_list[3], kernel_size=kernel_size_list[3], BN=True, NL=act_fn_dict['conv4'])
        self.maxpool4 = nn.MaxPool2d((2, 2))
        
        if self.dropout_list[1]!=0:
            self.dropout2 = nn.Dropout(dropout_list[1])

        self.conv5 = conv_block(no_kernel_list[3], no_kernel_list[4], kernel_size=kernel_size_list[4], BN=True, NL=act_fn_dict['conv5'])
        self.maxpool5 = nn.MaxPool2d((2, 2))
        
        self.fc1_in_features = get_fc_in(self.input_dim, kernel_size_list, no_kernel_list)
        
        self.fc1 = fc_block(self.fc1_in_features, fc1_nodes , NL=act_fn_dict['fc1'])
        
        if self.dropout_list[2]!=0:
            self.dropout3 = nn.Dropout(dropout_list[2])
        
        self.fc2 = nn.Linear(fc1_nodes, no_classes)
    
    
    def forward(self, x):
        if x.shape[2]!=self.input_dim:
            print("input dim not matched")
            return
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        
        if self.dropout_list[0]!=0:
            x = self.dropout1(x)
        
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.conv4(x)
        x = self.maxpool4(x)
        
        if self.dropout_list[1]!=0:
            x = self.dropout2(x)
        
        x = self.conv5(x)
        x = self.maxpool5(x)
        
        x = x.view(x.shape[0], -1)
        
        x = self.fc1(x)
        if self.dropout_list[2]!=0:
            x = self.dropout3(x)
        
        x = self.fc2(x)
        
        return x

In [13]:
model = CNN_5layer(kernel_size_list, no_kernel_list, act_fn_dict, dropout_list, fc1_nodes, no_classes)

In [14]:
model = model.to(device)

In [15]:
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
scheduler = StepLR(optimizer, step_size=10, gamma=lr_schedule)
criterion = nn.CrossEntropyLoss()

In [16]:
# load the saved last checkpoint
best_trained_model, optimizer, start_epoch, valid_loss, valid_acc, valid_loss_min = load_ckp(best_ckp_path, model, optimizer)

In [17]:
class_names = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

In [18]:
best_trained_model.eval()
test_loss = 0
correct = 0
best_loss = 1

example_images = []
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # Load the input features and labels from the test dataset
        data, target = data.to(device), target.to(device)

        # Make predictions: Pass image data from test dataset, make predictions about class image belongs to (0-9 in this case)
        output = best_trained_model(data)

        # Compute the loss sum up batch loss
        #test_loss += criterion(output, target, reduction='sum').item()
        
        loss = criterion(output, target)
        
        #pred = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1],target).view(-1)
        pred = torch.max(F.softmax(output, dim=1), dim=1)[1]
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        
        
        # Get the index of the max log-probability
        #pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(target.view_as(pred)).sum().item()
        #correct += torch.sum(pred).item()
        print(class_names[pred[0].item()]," ", class_names[target[0]])
        # WandB – Log images in your test dataset automatically, along with predicted and true labels by passing pytorch tensors with image data into wandb.Image
        example_images.append(wandb.Image(
            data[0], caption="Pred: {} Truth: {}".format(class_names[pred[0].item()], class_names[target[0]])))

# WandB – wandb.log(a_dict) logs the keys and values of the dictionary passed in and associates the values with a step.
# You can log anything by passing it to wandb.log, including histograms, custom matplotlib objects, images, video, text, tables, html, pointclouds and other 3D objects.
# Here we use it to log test accuracy, loss and some test images (along with their true and predicted labels).


Reptilia   Amphibia
Amphibia   Amphibia
Mollusca   Amphibia
Amphibia   Amphibia
Animalia   Animalia
Reptilia   Animalia
Animalia   Animalia
Arachnida   Arachnida
Arachnida   Arachnida
Arachnida   Arachnida
Mammalia   Aves
Mammalia   Aves
Aves   Aves
Fungi   Fungi
Plantae   Fungi
Fungi   Fungi
Arachnida   Insecta
Reptilia   Insecta
Reptilia   Insecta
Aves   Mammalia
Mammalia   Mammalia
Arachnida   Mammalia
Fungi   Mollusca
Animalia   Mollusca
Mollusca   Mollusca
Plantae   Plantae
Plantae   Plantae
Fungi   Plantae
Insecta   Plantae
Mammalia   Reptilia
Reptilia   Reptilia
Reptilia   Reptilia


In [19]:
wandb.log({"Test Set Examples with Predictions": example_images})